# Modélisation du taux de réussite et de la mention

Ce notebook est consacré à la **modélisation du taux de réussite et du taux de mention au baccalauréat**. Les données exploitées proviennent du notebook exploratoire `01_exploratoires.ipynb`, où elles ont été nettoyées, enrichies et mises à disposition pour l’analyse statistique et la modélisation.

---

## 1. Données disponibles

Les informations collectées et traitées incluent :

### a) Lycées et leur géolocalisation
- Informations techniques sur les établissements :  
  - Nom du lycée  
  - Département et région  
  - Secteur (public/privé)  
- Effectifs des élèves présents au bac, selon les différentes filières  
- Taux de réussite et de mention par filière  
- Nombre de mentions détaillé à l’échelle agrégée  
- Indicateurs de **valeur ajoutée** pour le taux de réussite et le taux de mention  

### b) Bibliothèques
- Localisation géographique des bibliothèques à proximité des lycées  
- Mesures de proximité, par exemple l’existence d’une bibliothèque dans un rayon de 100 m et le nombre de bibliothèques dans ce rayon  

### c) Indice de Position Sociale (IPS)
L’IPS renseigne sur la **composition sociale des élèves au sein de l’établissement** et s’inspire directement de la théorie bourdieusienne :  
- À partir de la PCS (catégorie socioprofessionnelle) des parents d’élèves, on déduit une **position sociale moyenne par lycée**  
- **Interprétation sociologique** : plus l’IPS est élevé, plus la position sociale moyenne des élèves est élevée, et plus ces derniers sont supposés mieux réussir au bac  

---

## 2. Variables d’intérêt

### a) Variables explicatives
- `ips` : indice de position sociale moyen du lycée  
- `d_lycee` : département du lycée  
- `s_lycee` : secteur du lycée (public/privé)  
- `existe_prepa` : présence de classes préparatoires dans le lycée (booléen)  
- `existe_biblio_100` : existence d’une bibliothèque dans un rayon de 100 m autour du lycée (booléen)  
- `n_biblio_100m` : nombre de bibliothèques dans un rayon de 100 m  
- Autres variables contextuelles pertinentes (ex. filière dominante, effectifs totaux, etc.)  

### b) Variable cible
- `taux_reussite` : proportion d’élèves ayant réussi au baccalauréat dans le lycée  

---

## 3. Modèle statistique adopté

Pour modéliser le **taux de réussite**, nous utilisons une **régression linéaire log-niveau** :  

$$
\begin{align*}
\log(\text{taux\_reussite}) &= \beta_0 
+ \beta_1 \cdot ips 
+ \beta_2 \cdot d\_lycee \\
&+ \beta_3 \cdot s\_lycee 
+ \beta_4 \cdot existe\_prepa 
+ \beta_5 \cdot existe\_biblio\_100 \\
&+ \beta_6 \cdot n\_biblio\_100m
\end{align*}
$$

### Pourquoi une transformation logarithmique ?
- La **log-transformation de la variable cible** permet de stabiliser la variance et de rendre les résidus plus proches d’une distribution normale  
- Cela facilite l’interprétation des coefficients : un coefficient \(\beta_i\) mesure approximativement l’effet **en pourcentage** d’une variation d’une unité de la variable explicative sur le taux de réussite  

### Interprétation attendue
- `β1` (IPS) : impact de la position sociale moyenne sur le taux de réussite  
- `β4` (existe_prepa) : effet de la présence d’une classe préparatoire  
- `β5` et `β6` (bibliothèques) : influence de l’accès aux bibliothèques sur la réussite  

---

Ce modèle constitue une première étape pour comprendre **les déterminants sociaux et institutionnels de la réussite au bac** et servira de base pour des modèles plus avancés (interaction entre variables, variables catégorielles, etc.).

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
data_final = pd.read_csv("data/data_final.csv")
X = data_final[["ips", "d_lycee", "s_lycee", "existe_prepa", "existe_biblio_100", "n_biblio_100m"]]
y = data_final["taux_reussite"]

y_log = np.log(y)
X = sm.add_constant(X)

In [ ]:
model = sm.OLS(y_log, X).fit()
print(model.summary())